In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
from bs4 import BeautifulSoup
import requests
import html2text

In [ ]:
sel_url = "https://siccode.com/business/"
wiki_url = "https://en.wikipedia.org/wiki/"

company_list = ["LinkedIn","Lynda.com","ZipRecruiter Inc","Indeed.com","SlideShare","Glassdoor","Connectifier"]

company_dict = {}

In [ ]:
# Selennium Portion
driver = webdriver.Chrome('/Users/kkannapp/Downloads/chromedriver')

for company in company_list:
    company_dict[company] = {}
    driver.get(sel_url+re.sub(r'\s+|\.', '-',company))
    try:
        companyCode = driver.find_element_by_xpath("/html/body/div[1]/div/section[2]/div/div/div[2]/div[2]/div/div/a[2]").text
        naics_code = re.findall(r'\d+', companyCode)
        company_dict[company]['naics'] = naics_code[0]
    except:
        company_dict[company]['naics'] = None
    time.sleep(2)

In [ ]:
# LinkedIn Portion
driver = webdriver.Chrome('/Users/kkannapp/Downloads/chromedriver')
linkedin_url = "https://www.linkedin.com/company/"

driver.get("https://www.linkedin.com")
driver.implicitly_wait(6)
driver.find_element_by_xpath("/html/body/nav/section[2]/form/div[1]/div[1]/input").send_keys(userid)
driver.find_element_by_xpath("/html/body/nav/section[2]/form/div[1]/div[2]/input").send_keys(password)
driver.find_element_by_xpath("/html/body/nav/section[2]/form/div[2]/button").click()

for company in company_list:
    driver.get(linkedin_url+re.split(r'\s+',re.sub(r'\.', '-',company))[0])
    try:
        companyCode = driver.find_element_by_xpath("/html/body/div[5]/div[4]/div[3]/div/div[2]/section/div/div/div[2]/div[2]/div/div[2]/a/span").text
        hc_number = re.findall(r'\d+',companyCode.replace(',', ''))
        company_dict[company]['headcount'] = hc_number
    except:
        company_dict[company]['headcount'] = None
    time.sleep(2)

In [ ]:
# Wikipedia Portion
h = html2text.HTML2Text()
h.ignore_links = True

for company in company_list:
    url = requests.get(wiki_url+(re.split(r'\s+',company)[0])).text
    soup = BeautifulSoup(url,'lxml')
    table = soup.find('table',{'class':'infobox vcard'})
    exceptional_row_count = 0
    for tr in table.find_all('tr'):
        if tr.find('th'):
            company_dict[company][h.handle(tr.find('th').text).strip().replace('\n','')] = h.handle(tr.find('td').text).strip().replace('\n',' ')
        else:
            # the first row Logos fall here
            exceptional_row_count += 1
    paragraph = soup.find("div",{'class':'mw-content-ltr'})
    pgtxt = paragraph.find_all('p')[1:3]
    for i in range(len(pgtxt)):
        company_dict[company]['pg_'+str(i)] = h.handle(str(pgtxt[i])).strip().replace('\n',' ')

In [ ]:
company_dict

In [ ]:
import json
with open('result_kk.json', 'w') as fp:
    json.dump(company_dict, fp)

In [ ]:
import json
import pandas as pd
import numpy as np
with open('result_kk.json', 'r') as f:
    docs = json.loads(f.read())

In [ ]:
key_list = []
for key in docs:
    k = list(docs.get(key).keys())
    key_list.append(k)

In [ ]:
flat_list = [item for sublist in key_list for item in sublist]
from collections import Counter
count_keys = Counter(flat_list)

In [ ]:
sorted(count_keys.items(), key=lambda pair: pair[1], reverse=True)

In [ ]:
df = pd.DataFrame.from_dict(docs,orient='index')
df.reset_index(level=0,inplace=True)
#pd.wide_to_long(df, i="index")

In [ ]:
#df.columns[1:]
df_lng = pd.melt(df, id_vars='index',value_vars = df.columns[1:])
df_lng.columns = ['company','attribute','value']
df_lng.tail(30)

In [ ]:
import pandas_dedupe
clustered_df = pandas_dedupe.dedupe_dataframe(df_lng,['attribute'])

In [ ]:
clustered_df = pd.read_csv('./dedupe_files/attribute_dedupe.csv')
clustered_df.groupby('cluster id')['attribute'].apply(lambda x: list(np.unique(x)))

Results are ok, some distance between the words are large - so manually correct a few (oracle job): notably

32 - 8, 21 - 5

In [ ]:
clustered_df['cluster id'].replace({32: 8, 21: 5},inplace=True)

In [ ]:
clustered_df.groupby('cluster id')['attribute'].apply(lambda x: list(np.unique(x)))

In [ ]:
clustered_df.tail(10)

In [ ]:
# Percent missingness
missing_df = clustered_df.groupby('cluster id')['value'].agg({'null_cnt': lambda x: x.isnull().sum()}).join(clustered_df.groupby('cluster id')['value'].size()).reset_index()

In [ ]:
# Remove clusters where missingness is above 70%:
missing_df['perc_miss'] = missing_df['null_cnt']/missing_df['value']
remove_list = missing_df[missing_df.perc_miss >=0.7]['cluster id'].to_list()

In [ ]:
filtered_df = clustered_df[~clustered_df['cluster id'].isin(remove_list)]

In [ ]:
final_df = filtered_df[~((filtered_df[['company', 'cluster id']].duplicated(keep=False)) & (filtered_df.isnull().any(axis=1)))].reset_index()

In [ ]:
final_df.head()
final_df.drop(['confidence','index'],axis=1,inplace=True)
final_df.columns = ['company','attribute','value','cluster_id']

In [ ]:
final_df.head()

In [ ]:
cl_map = final_df.groupby('cluster_id')['attribute'].apply(lambda x: list(np.unique(x)))
st_dict = {k:cl_map[k][0] for k in cl_map.index}

In [ ]:
final_df['std_attribute'] = final_df['cluster_id'].map(st_dict)

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('./clean_data.csv', columns=['company','std_attribute','value'], index=False)